<a href="https://colab.research.google.com/github/ciepielajan/Multi-Class-Classification-NLP/blob/main/PREDICT%20DEVELOP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PREDICT DEVELOP

#### loading the necessary files to predict in develop

In [6]:
# model
# https://drive.google.com/file/d/1LQzf7_SBpmee41X7ecupKFMGb8n9LXLk/view?usp=sharing
!gdown --id "1LQzf7_SBpmee41X7ecupKFMGb8n9LXLk" -O "develop_model.h5"

# config
# https://drive.google.com/file/d/11JehTWT6J_oHyVBAcdZtq8PRJ50pRKRz/view?usp=sharing
!gdown --id "11JehTWT6J_oHyVBAcdZtq8PRJ50pRKRz"  -O "develop_config.pickle"

Downloading...
From: https://drive.google.com/uc?id=1LQzf7_SBpmee41X7ecupKFMGb8n9LXLk
To: /content/develop_model.h5
11.8MB [00:00, 70.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=11JehTWT6J_oHyVBAcdZtq8PRJ50pRKRz
To: /content/develop_config.pickle
100% 451k/451k [00:00<00:00, 65.9MB/s]


In [7]:
from keras.models import load_model
model = load_model('develop_model.h5')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 7, 100)            976800    
_________________________________________________________________
flatten (Flatten)            (None, 700)               0         
_________________________________________________________________
dense (Dense)                (None, 7)                 4907      
Total params: 981,707
Trainable params: 981,707
Non-trainable params: 0
_________________________________________________________________


In [8]:
import pickle
with open('develop_config.pickle', 'rb') as f:
  develop_config = pickle.load(f)

develop_config

{'labelencoder': LabelEncoder(),
 'maxlen': 7,
 'tokenizer': <keras_preprocessing.text.Tokenizer at 0x7f8e74759310>}

In [9]:
labelencoder = develop_config["labelencoder"]
maxlen = develop_config["maxlen"]
tokenizer = develop_config["tokenizer"]

In [10]:
import re
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [11]:
def process_text(sentence):
    sentence = re.sub('[A-Za-z0-9]+@[a-zA-z].[a-zA-Z]+', '', sentence)  # maile
    sentence = re.sub('(http[s]*:[/][/])[a-zA-Z0-9]+', '', sentence)  # linki
    sentence = re.sub(r"<[^>]+>", " ", sentence) # remove html tag
    sentence = re.sub(r"[^a-zA-Z\s]", "", sentence)  # remove punctuations and numbers   
    sentence = re.sub(r"\b[A-Za-z]{1}\b", "", sentence)  # remove single characters

    sentence = re.sub("^\s+|\s+$", "", sentence, flags=re.UNICODE) # Remove spaces both in the BEGINNING and in the END of a string:
    sentence = " ".join(re.split("\s+", sentence, flags=re.UNICODE))  # Remove ONLY DUPLICATE spaces:
    sentence = sentence.lower()
    
    return sentence

#### Predict

In [12]:
# wszysktie etykiety
print(labelencoder.classes_.tolist())

['AddToPlaylist', 'BookRestaurant', 'GetWeather', 'PlayMusic', 'RateBook', 'SearchCreativeWork', 'SearchScreeningEvent']


In [24]:
# x = "PLAY <dasf>M</html> my 32 favorite music"
# x = "find my favorite book"
# x = "check the 'Red Hot Chili Peppers' discography"
# x = "play the 'Red Hot Chili Peppers' discography"
x = "how will its weather today" #@param {type:"string"}

x = process_text(x)
x = tokenizer.texts_to_sequences([x])
x = pad_sequences(x, padding="post", truncating="post", maxlen=maxlen)
predicted_lstm_val = np.argmax(model.predict(x), axis=-1)
y_pred = labelencoder.inverse_transform(predicted_lstm_val)[0]
y_pred

'GetWeather'